<a href="https://colab.research.google.com/github/Junaeid-Shoaib/AI_Agents/blob/main/Vision_Agent_Auto_with_previous_version_of_smolagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CHANGED: pin to the version compatible with the auto-instrumentor
!pip install --quiet "smolagents==1.20.0"

# keep your telemetry + Langfuse stack
!pip install --quiet smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents langfuse

# optional: align hub/client libs (harmless if already latest)
!pip install --quiet -U huggingface_hub openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.

In [ ]:
from huggingface_hub import login
login()


In [ ]:
from PIL import Image
import requests
from io import BytesIO

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",  # Joker image
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"     # Joker image
]

images = []
for url in image_urls:
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers, timeout=30)
    resp.raise_for_status()
    images.append(Image.open(BytesIO(resp.content)).convert("RGB"))


In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('secretKey')


In [ ]:
import os
from google.colab import userdata
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('pk')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('sk')
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # EU region

from langfuse import get_client
langfuse = get_client()

if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")


Langfuse client is authenticated and ready!


In [ ]:
# keep the auto-instrumentor ON
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
SmolagentsInstrumentor().instrument()

from smolagents import CodeAgent, OpenAIServerModel

model = OpenAIServerModel(model_id="gpt-4o", temperature=0.0)
agent = CodeAgent(tools=[], model=model, max_steps=8, verbosity_level=2)

# sanity check versions (smolagents should print 1.20.0)
import smolagents, huggingface_hub
print("smolagents:", smolagents.__version__)
print("huggingface_hub:", huggingface_hub.__version__)


smolagents: 1.20.0
huggingface_hub: 0.34.4


In [ ]:
prompt = """
Describe the costume and makeup that the comic character in these photos is wearing.
Then answer clearly: Is this guest The Joker or Wonder Woman?
"""

response = agent.run(prompt, images=images)
print(response)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing.                            │
│ Then answer clearly: Is this guest The Joker or Wonder Woman?                                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
The character in the photos is wearing a costume and makeup that includes:                                         
                                                                                                                   
- **Makeup**: White face paint with exaggerated red lips and dark eye makeup.                                      
- **Hair**: Green hair.                                                                                            
- **Costume**: A purple suit with a yellow shirt and a purple tie or bow.                                          
                                                                                                                   
This description matches the appearance of **The Joker**.                                                          
                                                                                                                   
Final answer: The guest is The Joker.                                                                              

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            The character in the photos is wearing a costume and makeup that includes:

- **Makeup**: White face paint with exaggerated red lips and dark eye makeup.
- **Hair**: Green hair.
- **Costume**: A purple suit with a yellow shirt and a purple tie or bow.

This description matches the appearance of **The Joker**. 

Final answer: The guest is The Joker.</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 5.03 seconds| Input tokens: 2,930 | Output tokens: 81]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Description of the character's appearance                                                                      
  makeup = "White face paint with exaggerated red lips and dark eye makeup."                                       
  hair = "Green hair."                                                                                             
  costume = "Purple suit with a yellow shirt and a purple tie or bow."                                             
                                                                                                                   
  # Determine the character                                                                                        
  character = "The Joker"                                                                                          
                                                                                                                   
  # Print the description and final answer                                                                         
  print(f"Makeup: {makeup}")                                                                                       
  print(f"Hair: {hair}")                                                                                           
  print(f"Costume: {costume}")                                                                                     
  final_answer(character)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Makeup: White face paint with exaggerated red lips and dark eye makeup.
Hair: Green hair.
Costume: Purple suit with a yellow shirt and a purple tie or bow.

Final answer: The Joker

[Step 3: Duration 13.49 seconds| Input tokens: 12,691 | Output tokens: 292]

The Joker
